# glacier3dviz workflow #1: Hi-res simulation and data creation  
This notebook can be used to run high-resolution future simulations of glaciers with the [OPEN GLOBAL GLACIER MODEL(OGGM)](https://oggm.org/). Each single glacier run is dynamically calibrated with OGGM's [dynamic spinup](https://docs.oggm.org/en/stable/dynamic-spinup.html) and can then be forced with different global climate(GCM) model runs.
The aim of this notebook is to prepare data for a 3D animation with the [glacier3dviz](https://glacier3dviz.oggm.org/) tool. Therefor the resulting flowline data of the simulaton is redistributed into spatially distributed data which then can be animated on the actual topography in which the glacier is embedded. 


## Imports & Setup

In [ ]:
from oggm import cfg, utils, workflow, tasks, DEFAULT_BASE_URL
from oggm.sandbox import distribute_2d
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import os
import xarray as xr
from oggm.shop import gcm_climate
from os.path import join
import pandas as pd
import shutil

This series of notebooks is written to be used on a HPC cluster with the help of jupytext and slurm. When run as a notebook, it only executes the workflow as a test-run for one glacier.

In [ ]:
# Function to detect if we're running in a Jupyter notebook
def check_if_notebook():
    try:
        shell_name = get_ipython().__class__.__name__
        if shell_name == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or JupyterLab
        elif shell_name in ['TerminalInteractiveShell', 'InteractiveShell']:
            return False  # IPython terminal or other interactive shells
        else:
            # Fallback or default behavior for unidentified environments
            return False
    except NameError:
        return False      # Not in IPython, likely standard Python interpreter

# Use this to conditionally execute tests/debugging
if check_if_notebook():
    is_notebook = True
else:
    is_notebook = False

In [ ]:
# we always need to initialzie and define a working directory
cfg.initialize(logging_level='WARNING')

# if we execute this script as a notebook, the working dir of OGGM will be located in the same folder, named 'working_dir_testing'
if is_notebook:
    working_dir = 'working_dir_testing'
# if we execute this script via slurm, the working dir of OGGM will be copied to the same folder after the slurm run and will be named 'working_dir'
else:
    working_dir = os.path.join(os.environ["OGGM_WORKDIR"],
                               'working_dir'
    )

cfg.PATHS['working_dir'] = working_dir

# use multiprocessing when running the script on the cluster
if not is_notebook:
    cfg.PARAMS['use_multiprocessing'] = True

In this example script the glaciers for the [goodbye glaciers website](https://goodbye-glaciers.info) are listed, but you can choose any other glacier that are part of the Randolph's Glacier Invetory v6 (this glacier explorer can help you find your glaciers' RGI6 IDs: [GLIMS GLACIER EXPLORER](https://www.glims.org/maps/glims)).
If you want to animate neighbouring/nearby glaciers together in one animation, you can define a glacier complex in the `merge_glaciers` dictionary. A complex is defined by its name(used for file naming) and a list of the RGI-IDs that should be part of this complex. The glaciers of a complex also have to be part of the `rgi_ids`-list.

In [ ]:
rgi_ids = [
    'RGI60-11.00897', # Hintereisferner
    'RGI60-11.00106', # Pasterze
    'RGI60-11.01238', # Rhone
    'RGI60-11.03887', # Marmolada
    'RGI60-11.03671', # Gebroulaz
    'RGI60-11.00593' # Seekarlesferner
]
# if no glaciers should be merged, set `merge_glaciers` to `None`
merge_glaciers = None

## Preparing the gdirs and initializing the glaciers

In [ ]:
# loading gdirs from preprocessed directories
prepro_base_url_L0 = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L1-L2_files/elev_bands/'
gdirs = workflow.init_glacier_directories(rgi_ids,
                                          from_prepro_level=0,
                                          prepro_base_url=prepro_base_url_L0,
                                          prepro_border=80,  # could be 10, 80, 160 or 240
                                          reset=True,
                                          force=True,
                                         )

In [ ]:
if is_notebook:
    gdirs = [gdirs[0]]

#### in the following cell the gridsize(which equals the resolution of the glacier in meters) of the simulation can be set with `cfg.PARAMS['fixed_dx']`. Here it is set to 30 meters, which is already a high resolution for a big glacier in the European Alps(e.g. Aletsch, Hintereisferner..).

In [ ]:
# define the border, we keep the default here
cfg.PARAMS['border'] = cfg.PARAMS['border']

# set the method for determining the local grid resolution
cfg.PARAMS['grid_dx_method'] = 'fixed'  # The default method is 'square', which determines the grid spacing (dx) based on the glacier's outline area.
cfg.PARAMS['fixed_dx'] = 30  # This allows setting a specific resolution in meters. It's applicable only when grid_dx_method is set to 'fixed'.
# set the DEM source to use
source = 'COPDEM30'  # we stick with the OGGM default

# this task adds the DEM and defines the local grid
workflow.execute_entity_task(tasks.define_glacier_region, gdirs,
                             source=source);

### choose type of flowline to use: 'elevation_band' or 'centerline' ([more info on flowline types](https://docs.oggm.org/en/latest/flowlines.html))

In [ ]:
flowline_type_to_use = 'elevation_band'  # you can also select 'centerline' here

if flowline_type_to_use == 'elevation_band':
    elevation_band_task_list = [
        tasks.simple_glacier_masks,
        tasks.elevation_band_flowline,
        tasks.fixed_dx_elevation_band_flowline,
        tasks.compute_downstream_line,
        tasks.compute_downstream_bedshape,
    ]

    for task in elevation_band_task_list:
        workflow.execute_entity_task(task, gdirs);

elif flowline_type_to_use == 'centerline':
    # for centerline we can use parabola downstream line
    cfg.PARAMS['downstream_line_shape'] = 'parabola'

    centerline_task_list = [
        tasks.glacier_masks,
        tasks.compute_centerlines,
        tasks.initialize_flowlines,
        tasks.catchment_area,
        tasks.catchment_intersections,
        tasks.catchment_width_geom,
        tasks.catchment_width_correction,
        tasks.compute_downstream_line,
        tasks.compute_downstream_bedshape,
    ]

    for task in centerline_task_list:
        workflow.execute_entity_task(task, gdirs);
    
else:
    raise ValueError(f"Unknown flowline type '{flowline_type_to_use}'! Select 'elevation_band' or 'centerline'!")

### massbalance calibration ([more info](https://tutorials.oggm.org/stable/notebooks/tutorials/massbalance_calibration.html))

In [ ]:
# define the climate data to use, we keep the default
cfg.PARAMS['baseline_climate'] = cfg.PARAMS['baseline_climate']

# add climate data to gdir
workflow.execute_entity_task(tasks.process_climate_data, gdirs);

# the default mb calibration
workflow.execute_entity_task(tasks.mb_calibration_from_geodetic_mb,
                             gdirs,
                             informed_threestep=True,  # only available for 'GSWP3_W5E5'
                            );

# glacier bed inversion
workflow.execute_entity_task(tasks.apparent_mb_from_any_mb, gdirs);
workflow.calibrate_inversion_from_consensus(
    gdirs,
    apply_fs_on_mismatch=True,
    error_on_mismatch=True,  # if you running many glaciers some might not work
    filter_inversion_output=True,  # this partly filters the overdeepening due to
    # the equilibrium assumption for retreating glaciers (see. Figure 5 of Maussion et al. 2019)
    volume_m3_reference=None,  # here you could provide your own total volume estimate in m3
);

# finally create the dynamic flowlines
workflow.execute_entity_task(tasks.init_present_time_glacier, gdirs);

### dynamic spinup ([more info](https://tutorials.oggm.org/stable/notebooks/tutorials/dynamical_spinup.html))

In [ ]:
cfg.PARAMS['store_fl_diagnostics'] = True

# set the ice dynamic solver depending on the flowline-type
if flowline_type_to_use == 'elevation_band':
    cfg.PARAMS['evolution_model'] = 'SemiImplicit'
elif flowline_type_to_use == 'centerline':
    cfg.PARAMS['evolution_model'] = 'FluxBased'
else:
    raise ValueError(f"Unknown flowline type '{flowline_type_to_use}'! Select 'elevation_band' or 'centerline'!")

# get the start and end year of the selected baseline
y0 = gdirs[0].get_climate_info()['baseline_yr_0']
ye = gdirs[0].get_climate_info()['baseline_yr_1'] + 1  # run really to the end until 1.1.


# 'dynamic' initialisation, including dynamic mb calibration
dynamic_spinup_start_year = 1979
minimise_for = 'area'  # other option would be 'volume'
for gdir in gdirs:
    workflow.execute_entity_task(
        tasks.run_dynamic_melt_f_calibration, [gdir],
        err_dmdtda_scaling_factor=0.2,  # by default we reduce the mass balance error for accounting for
        # corrleated uncertainties on a regional scale
        ys=dynamic_spinup_start_year, ye=ye,
        kwargs_run_function={'minimise_for': minimise_for,
                                'precision_percent': 1.3} if gdir.rgi_id == 'RGI60-11.03887' else {'minimise_for': minimise_for},
        ignore_errors=True,
        kwargs_fallback_function={'minimise_for': minimise_for},
        output_filesuffix='_spinup_historical',
        
    );

In [ ]:
# look at the statistics if everything is working
opath = os.path.join(cfg.PATHS['working_dir'], 'glacier_statistics.csv')
df_statistics = utils.compile_glacier_statistics(gdirs, path=opath)

In [ ]:
if is_notebook:
    # if all none all worked without an error
    print(df_statistics['error_task'])

## Download and process GCM data - 2 options:
#### - set `temperature_scenarios` to `False` to use ssp-based clustering of the scenarios 
#### - set `temperature_scenarios` to `True` to use warming-based clustering of the scenarios 
#### - the ssp-scenarios and temperature scenarios can be defined further down

In [ ]:
# choose one option by setting the boolean:
# - True: wariming-based scenario clustering
# - False: ssp-based scenario clustering
temperature_scenarios = True

In [ ]:
# load CMIP5 + CMIP6 metadata
gcms_cmip6 = pd.read_csv('/home/www/oggm/cmip6/all_gcm_list.csv', index_col=0)
gcms_cmip5 = pd.read_csv('/home/www/oggm/cmip5-ng/all_gcm_list.csv', index_col=0)

### Option I: SSP scenarios taken from CMIP6
#### - If you choose this option, define the ssp scenarios that will be used in the third line (e.g. `scenarios = ['ssp126', 'ssp585']`)

In [ ]:
if not temperature_scenarios: 
    # set the ssp scenarios here
    scenarios = ['ssp126', 'ssp585']
    
    def get_models_per_scenario(scenario):
        return np.unique(gcms_cmip6[gcms_cmip6.ssp == scenario].gcm.values)
    
    gcms_per_scenario = {}
    for scenario in scenarios:
        gcms_per_scenario[scenario] = get_models_per_scenario(scenario)
        # if is_notebook:
        #     gcms_per_scenario[scenario] = [gcms_per_scenario[scenario][0]]

    # iterate through the scenarios        
    for scenario in gcms_per_scenario:
        for gcm in gcms_per_scenario[scenario]:
            rid = f'{gcm}_{scenario}'
    
            select_gcm = np.array([g.upper() for g in gcms_cmip6.gcm]) == gcm.upper()
            select_ssp = gcms_cmip6.ssp == scenario
            selected_run = gcms_cmip6[select_gcm & select_ssp]
            ft = selected_run[selected_run['var'] == 'tas'].path.values[0]
            fp = selected_run[selected_run['var'] == 'pr'].path.values[0]
    
            ft = utils.file_downloader(ft)
            fp = utils.file_downloader(fp)
            
            # bias correct them
            workflow.execute_entity_task(gcm_climate.process_cmip_data, gdirs,
                                         year_range=('2000', '2019'),
                                         filesuffix=rid,  # recognize the climate file for later
                                         fpath_temp=ft,  # temperature projections
                                         fpath_precip=fp,  # precip projections
                                         );

### Option II: Selection of Model runs from the CMIP5 and CMIP6 that show a certain warming towards the end of the century
#### - define the temperature scenarios that will be used in the third code line (e.g. `scenarios = [1.5, 2.7]`)
#### - define the temperature margins that will be used in the fourth code line (e.g. `temp_margin = 0.25`)
#### this will select all GCM runs, from the ones we have available(CMIP5 and CMIP6), for which the "preindustrial-2100" global warming value is within the given temperature margin around the given temperature level.

In [ ]:
if temperature_scenarios:
    # set the temperatures and the temperature margin
    scenarios = [1.5, 2.7] # °C warming from pre-industrial to the year 2100
    temp_margin = 0.25 # +/- °C temperature difference from the given scenario value that is still accepted for this scenario
    
    # load the data containing the temperature difference between pre-industrial and the modelled year 2100
    _file_cmip5 = '/home/www/oggm/oggm-standard-projections/analysis_notebooks/Global_mean_temp_deviation_2071_2100_2081_2100_2271_2300_2281_2300_rel_1850_1900_cmip5_gcms_ipcc_ar6_def.csv'
    _file_cmip6 = '/home/www/oggm/oggm-standard-projections/analysis_notebooks/Global_mean_temp_deviation_2071_2100_2081_2100_2271_2300_2281_2300_rel_1850_1900_cmip6_gcms_ipcc_ar6_def.csv'
    pd_cmip5_temp_ch_2100 = pd.read_csv(_file_cmip5, index_col=0)
    pd_cmip5_temp_ch_2100['cmip'] = 'CMIP5'
    
    pd_cmip6_temp_ch_2100 = pd.read_csv(_file_cmip6, index_col=0)
    pd_cmip6_temp_ch_2100['cmip'] = 'CMIP6'
    
    pd_cmip_temp_ch_2100 = pd.concat([pd_cmip5_temp_ch_2100, pd_cmip6_temp_ch_2100])
    pd_cmip_temp_ch_2100.index = list(map(lambda x: x.upper(), pd_cmip_temp_ch_2100.index))
    
    ## chosen temp...
    def get_models_from_temp(temp, lower_limit, upper_limit):
        pi_l = temp + lower_limit
        pi_u = temp + upper_limit
        pd_cmip_sel = pd_cmip_temp_ch_2100.loc[(pd_cmip_temp_ch_2100['global_temp_ch_2071-2100_preindustrial']>=pi_l)&
        (pd_cmip_temp_ch_2100['global_temp_ch_2071-2100_preindustrial']<=pi_u)]
        return pd_cmip_sel
    scenarios_with_gcm = {}
    for scenario in scenarios:
        scenario_name = '+' + str(scenario) + '°C'
        scenarios_with_gcm[scenario_name] = get_models_from_temp(scenario, -abs(temp_margin), +abs(temp_margin))

    for scenario in scenarios_with_gcm:
        df_gcm = scenarios_with_gcm[scenario]
        for index, gcm in df_gcm.iterrows():
            if gcm.cmip == 'CMIP5':
                select_gcm = np.array([g.upper() for g in gcms_cmip5.gcm]) == gcm.gcm
                select_rcp = gcms_cmip5.rcp == gcm.rcp
                selected_run = gcms_cmip5[select_gcm & select_rcp]
                ft = selected_run[selected_run['var'] == 'tas'].path.values[0]
                fp = selected_run[selected_run['var'] == 'pr'].path.values[0]
            elif gcm.cmip == 'CMIP6':
                select_gcm = np.array([g.upper() for g in gcms_cmip6.gcm]) == gcm.gcm
                select_rcp = gcms_cmip6.ssp == gcm.ssp
                selected_run = gcms_cmip6[select_gcm & select_rcp]
                ft = selected_run[selected_run['var'] == 'tas'].path.values[0]
                fp = selected_run[selected_run['var'] == 'pr'].path.values[0]
            else:
                raise ValueError(gcm.cmip)
    
            ft = utils.file_downloader(ft)
            fp = utils.file_downloader(fp)
            # bias correct them
            workflow.execute_entity_task(gcm_climate.process_cmip_data, gdirs,
                                         year_range=('2000', '2019'),
                                         filesuffix=index,  # recognize the climate file for later
                                         fpath_temp=ft,  # temperature projections
                                         fpath_precip=fp,  # precip projections
                                         );
    
    

    

## Projection runs 
### The following cells will exectue the actual OGGM model runs, 1 run for each GCM and each Glacier selected

In [ ]:
# these values have to be set to True, so that the flowline evolution over the projected period is saved for each timestep. This data is later on used for the redistribution and finally for the animation of the glaciers.
cfg.PARAMS['store_model_geometry'] = True
cfg.PARAMS['store_fl_diagnostics'] = True

In [ ]:
if temperature_scenarios:
    scenario_suffixes = {key: list(value.index) for key, value in scenarios_with_gcm.items()}
else:
    for ssp_scenario, gcms in gcms_per_scenario.items():
        gcms = gcms.astype(str)
        gcms_per_scenario[ssp_scenario] = np.char.add(gcms, f"_{ssp_scenario}").tolist()
        scenario_suffixes = gcms_per_scenario

    


#### We now run OGGM forced with various climate scenarios **starting from the end year of the historical spin-up run**:

In [ ]:
for scenario in scenario_suffixes.values():
    for scenario_suffix in scenario:
        workflow.execute_entity_task(tasks.run_from_climate_data, gdirs,
                                     climate_filename='gcm_data',  # use gcm_data, not climate_historical
                                     climate_input_filesuffix=scenario_suffix,  # use the chosen scenario
                                     init_model_filesuffix='_spinup_historical',  # this is important! Start from 2020 glacier
                                     output_filesuffix=scenario_suffix,  # recognize the run for later
                                    );

## Calculate the model median for each scenario
This function takes the different gcm runs for one glacier and calculates a median flowline (quantiles = 0.5)

In [ ]:
for scenario, suffix_list in scenario_suffixes.items():
    workflow.execute_entity_task(tasks.compute_fl_diagnostics_quantiles,
                             gdirs,
                             input_filesuffixes=suffix_list,
                             quantiles=0.5,
                             output_filesuffix=f'_median_{scenario}'
                             )

## Create redistributed data for each glacier and glacier complex
### The data that will be generated below is used for the creation of the animations in the next script.

`create_mini_ds()` is a helper function that allows to create a small version of our data for test purposeses.

In [ ]:
def mini_ds_generator(input_dir, output_dir):
    """
    Process .nc files in the input directory, reduce to first two timesteps,
    and save to the output directory, preserving the directory structure. 
    This so created mini datasets can be used for test purposes without moving large files.
    
    Args:
        input_dir (str): The source directory containing the .nc files.
        output_dir (str): The destination directory to save the processed .nc files.
    """
    # Walk through the directory structure of the input directory
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.nc'):  # Process only .nc files
                input_file_path = os.path.join(root, file)
                
                # Construct the relative path of the file
                relative_path = os.path.relpath(root, input_dir)
                output_folder = os.path.join(output_dir, relative_path)
                
                # Ensure the output folder exists
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)
                
                # Open the .nc file with xarray
                ds = xr.open_dataset(input_file_path)
                
                # Reduce the dataset to the first two timesteps along the 'time' dimension
                ds_reduced = ds.isel(time=slice(0, 2))
                
                # Define the output file path
                output_file_path = os.path.join(output_folder, file)
                
                # Save the reduced dataset as a new .nc file
                ds_reduced.to_netcdf(output_file_path)
                
                # Close the dataset
                ds.close()


### If you want to create mini-ds, which you can easily copy to your local workspace to test the animation settings: 
### set `create_mini_ds=True`

In [ ]:
create_mini_ds = True

# create animation data folder, if it exists already, keep using/overwriting it
redist_data_path = os.path.join(cfg.PATHS['working_dir'], 'redistributed_data')
os.makedirs(redist_data_path, exist_ok=True)
if create_mini_ds:
    mini_ds_path = os.path.join(cfg.PATHS['working_dir'], 'mini_ds')

In [ ]:
# This is to add a new topography to the file (smoothed differently)
workflow.execute_entity_task(distribute_2d.add_smoothed_glacier_topo,
                             gdirs)
    # This is to get the bed map at the start of the simulation
workflow.execute_entity_task(tasks.distribute_thickness_per_altitude,
                             gdirs)
    # This is to prepare the glacier directory for the interpolation (needs to be done only once)
workflow.execute_entity_task(distribute_2d.assign_points_to_band,
                             gdirs)

for scenario in scenario_suffixes:
    for gdir in gdirs:
        input_filesuffix = f'_median_{scenario}'
        workflow.execute_entity_task(distribute_2d.distribute_thickness_from_simulation,
                                     gdir, 
                                     input_filesuffix=input_filesuffix, 
                                     concat_input_filesuffix='_spinup_historical',
                                     fl_thickness_threshold=2,
                                     rolling_mean_smoothing=8,
                                     debug_area_timeseries=True,
                                     add_monthly=True,
                                     smooth_radius=48
                                    )
        # copy relevant data to a directory outside of the gdirs.
        rgi_dir = os.path.join(redist_data_path, gdir.rgi_id)
        os.makedirs(rgi_dir, exist_ok=True)
        shutil.copy(gdir.get_filepath('gridded_simulation', filesuffix=input_filesuffix), rgi_dir)
        

### If defined, the data of (neighboring) glaciers can also be merged to have the time evolution of several glaciers in one file.

In [ ]:
if not is_notebook:
    if merge_glaciers:
        # merge simulations
        for name in merge_glaciers:
            print(f'Merging {name}:')
            # get gdirs of current selection
            gdirs_merging = []
            for gdir in gdirs:
                if gdir.rgi_id in merge_glaciers[name]:
                    gdirs_merging.append(gdir)
        
            # merge each scenario
            for scenario in list(scenario_suffixes.keys()):
                print(f'   {scenario}')
                distribute_2d.merge_simulated_thickness(
                    gdirs_merging,  # the gdirs we want to merge
                    simulation_filesuffix=f'_median_{scenario}',  # the name of the simulationinput_filesuffix
                    output_filename=f'gridded_simulation_merged_{name}_median_{scenario}',
                    output_folder=redist_data_path,
                    years_to_merge=None,  # for demonstration I only pick some years, if this is None all years are merged
                    add_topography='COPDEM30',  # if you do not need topogrpahy setting this to False will decrease computing time
                    preserve_totals=True,  # preserve individual glacier volumes during merging
                    reset=True,
                    save_as_multiple_files=False,
                    )

In [ ]:
if create_mini_ds:
    mini_ds_generator(redist_data_path, mini_ds_path)
            
            

In [ ]:
print(f'FINISHED. All redistributed thickness files are saved at \'{redist_data_path}\'.')

## The data created above can now be used to create 3D animations. See [next tutorial](https://www.lipsum.com/). 